In [3]:
import numpy as np
np.random.seed(seed = None)

# SimInvDynNonPerish
This is the first version of the algorithm, without taking into acount perishabillity of the items

In [4]:
def get_demand(i):
    # in order to each time generate a different number we are going to use i as the date number 
    #np.random.seed(i)
    return np.random.poisson(4,1)

def SimInDynNonPerish (S,It,i):
    #determine demand
    dt = 0
    #order quantity
    Qt = 0
    fillrate = 0
    items_sold = 0
    fillrates = []
    for i in range (0,i):
        Qt = S-It 
        if Qt <0: #Because we cannot return items
            Qt = 0
        dt = get_demand(i)
        #print(demand)
        #set demand level if there is more demand than orders It=0
        if dt>It:
            items_sold = It
            It=0
        else:
            It = It-dt
            items_sold = dt
        if dt>0:
            fillrate = items_sold/dt
        else:
            fillrate = 1
        fillrates.append(fillrate)
        It = It + Qt
        
    return fillrates
        
        
    

In [5]:
fillrate_lst = SimInDynNonPerish(14,10,365)
np.mean(fillrate_lst)

array([0.99720483])

# Extension Perishabillity 

In [271]:
#S is order up to level, i is the time period, f is the fraction of fifo consumers
#Set stock of first day to 14
def determine_type_consumer(f,dt):
    #To determine the type of consumer we need the freaction of costumers of FIFO type f and the dt
    fifo_consumers = np.random.binomial(dt,f)
    return [fifo_consumers,dt-fifo_consumers]

def get_remaining_stock(s, fifo, lifo):
    Stock = s.copy()
    for i in range(len(Stock)):
        #print(i,len(Stock))
        #if fifo is greaer than s substract from fifo and set the longer to expire Stock to 0
        tmp_f = Stock[i]
        tmp_l = Stock[len(Stock) - i-1]
        if fifo > Stock[i]:
            Stock[i] = 0
            fifo = fifo - tmp_f
            
        else:
            Stock[i] = Stock[i] - fifo
            fifo = 0
            
        if lifo > Stock[- i-1]:
            Stock[-i-1] = 0
            lifo = lifo - tmp_l
            
        else:
            Stock[-i-1] = Stock[-i-1] - lifo
            lifo = 0
        
    return Stock
    
def SimInDynPerish (S,i,It=14,f=0.5, cw=100, cs=500):
    #determine demand
    dt = 0
    #order quantity
    Qt = 0
    fillrate = 0
    items_sold = 0
    #assume that in place 0 of the list is the most recent product and place 3 the oldest ones
    Stock=[It,0,0,0]
    fillrates = []
    #waste percent, list of percentage wast per day 
    waste = 0
    waste_sum = 0
    #get a count of total number of products sold 
    total_products = It
    #get Average cost per day
    avg_cost_day = []
    
    for i in range (0,i):
        #day start 
        #product shortage, updated daily
        waste = 0
        prod_short = 0
        #we order up to level S
        Qt = S - sum(Stock)
        
        if Qt <0: #Because we cannot return items
            Qt = 0
        #determine day demand
        dt = get_demand(i)
        total_products += dt
        #determine consumer type number 
        fifo,lifo = determine_type_consumer(f,dt)
        #print(demand)
        #set demand level if there is more demand than Stock
        if dt>= sum(Stock):
            items_sold = sum(Stock)
            prod_short = dt - sum(Stock)
            Stock = [0,0,0,0]
        else:
        #fifo consumers take fifo amount of the most recent products
        #and lifo consumers take lifo amount of the oldest products 
            Stock = get_remaining_stock(Stock,fifo,lifo)
            items_sold = dt
        
        if dt>0:
            fillrate = items_sold/dt
        else:
            fillrate = 1
        fillrates.append(fillrate)
        #waste, pop the remaing old products
        waste += Stock.pop()
        waste_sum+=waste
        #add order to stock
        Stock = [Qt] + Stock
        avg_cost_day.append(waste*cw + prod_short * cs)
        #print(waste,prod_short)
    total_waste_per = 100 - abs((waste_sum - total_products)/total_products) *100
    print(waste_sum, total_products)
    return (np.mean(fillrates),total_waste_per,np.mean(avg_cost_day))

In [287]:
SimInDynPerish(12,365)

[83] [1494]


(array([0.98145822]), array([5.55555556]), 91.23287671232876)